# INST326 — Week 8 Exercises: Inheritance & Polymorphism (Library Management)

**Focus (Week 8 only):** subclassing, method overriding, `super()`, polymorphism via common method names, and composition vs. inheritance decisions in a small codebase.

**Out of scope (Week 9+):** abstract base classes, interfaces/protocols, multiple inheritance/mixins, advanced design patterns, decorators beyond basics, context managers beyond prior weeks, dependency injection, property descriptors beyond simple use.

> Context: Use the Library Management domain—books, members, loans, fines—to complete the tasks. Stick to basic single inheritance and straightforward overrides.


### Starter Scaffold (Week‑8‑safe)

Below is minimal starter code from prior weeks, extended slightly for Week 8. Feel free to modify it for the exercises. Avoid Week 9+ topics.


In [ ]:
from __future__ import annotations
from dataclasses import dataclass
from datetime import datetime, timedelta
from typing import Dict, Optional, List

# --- Exceptions from Week 7 (basic) ---
class LibraryError(Exception): ...
class DuplicateBookError(LibraryError): ...
class OverdueLoanError(LibraryError): ...

# --- Base domain classes (no ABCs) ---
@dataclass
class Book:
    isbn: str
    title: str
    copies: int = 1

    def loan_period_days(self) -> int:
        """Default loan period for a generic book."""
        return 14

    def describe(self) -> str:
        return f"Book<{self.isbn}>: {self.title} (copies={self.copies})"

@dataclass
class Member:
    member_id: str
    email: str

    def max_concurrent_loans(self) -> int:
        return 5

    def describe(self) -> str:
        return f"Member<{self.member_id}>"

@dataclass
class Loan:
    isbn: str
    member_id: str
    due_date: datetime
    returned: bool = False

    def mark_returned(self) -> None:
        self.returned = True

class Catalog:
    def __init__(self):
        self._books: Dict[str, Book] = {}

    def add_book(self, book: Book) -> None:
        if book.isbn in self._books:
            raise DuplicateBookError(f"ISBN already exists: {book.isbn}")
        if book.copies < 0:
            raise ValueError("copies must be non-negative")
        self._books[book.isbn] = book

    def get_book(self, isbn: str) -> Optional[Book]:
        return self._books.get(isbn)

class LoanDesk:
    """Very small service; deliberately simple for Week 8 examples."""
    def __init__(self, catalog: Catalog):
        self.catalog = catalog
        self.loans: List[Loan] = []

    def checkout(self, member: Member, book: Book) -> Loan:
        # naive stock check
        if book.copies <= 0:
            raise LibraryError("no available copies")
        book.copies -= 1
        due = datetime.now() + timedelta(days=book.loan_period_days())
        loan = Loan(isbn=book.isbn, member_id=member.member_id, due_date=due)
        self.loans.append(loan)
        return loan

    def checkin(self, loan: Loan) -> None:
        if not loan.returned:
            b = self.catalog.get_book(loan.isbn)
            if b:
                b.copies += 1
            loan.mark_returned()


## 1) Subclass a Book type

Create a subclass `PrintedBook(Book)` that overrides `loan_period_days()` to 21 days and `describe()` to include the word 'Printed'.

In [7]:
# Your code here
from dataclasses import dataclass

@dataclass
class Book:
    isbn: str
    title: str
    copies: int = 1

    def loan_period_days(self) -> int:
        """Default loan period for a generic book."""
        return 14

    def describe(self) -> str:
        return f"Book<{self.isbn}>: {self.title} (copies={self.copies})"

class PrintedBook(Book):
    def loan_period_days(self) -> int:
        return 21

    def describe(self) -> str:
        return f"Printed{super().describe()}"

# Example:
pb = PrintedBook("111", "Intro to Python", 2)
pb.loan_period_days()  # 21
pb.describe()          # includes 'Printed'

'PrintedBook<111>: Intro to Python (copies=2)'

## 2) Another Book subtype

Create `EBook(Book)` that has an extra attribute `file_size_mb: float` (add to `__init__`), uses a 14‑day loan, and overrides `describe()` to show the size.

In [10]:
# Your code here

from dataclasses import dataclass

@dataclass
class Book:
    isbn: str
    title: str
    copies: int = 1

    def loan_period_days(self) -> int:
        """Default loan period for a generic book."""
        return 14

    def describe(self) -> str:
        return f"Book<{self.isbn}>: {self.title} (copies={self.copies})"


class EBook(Book):
    def __init__(self, isbn: str, title: str, copies: int = 1, file_size_mb: float = 0.0):
        super().__init__(isbn, title, copies)
        self.file_size_mb = file_size_mb
    
    def decribe(self) -> str:
        return f"Ebook<{self.isbn}>: {self.title} (copies={self.copies}, size={self.file_size_mb}MB)"

## 3) Override with super()

Create `AudioBook(Book)` with extra field `duration_min: int`. Override `describe()` to start with `super().describe()` and append `duration_min`.

In [11]:
# Your code here

from dataclasses import dataclass

@dataclass
class Book:
    isbn: str
    title: str
    copies: int = 1

    def loan_period_days(self) -> int:
        """Default loan period for a generic book."""
        return 14

    def describe(self) -> str:
        return f"Book<{self.isbn}>: {self.title} (copies={self.copies})"
class AudioBook(Book):
    def __init__(self, isbn: str, title: str, copies: int = 1, duration_minutes: int = 0):
        super().__init__(isbn, title, copies)
        self.duration_minutes = duration_minutes

    def describe(self) -> str:
        return f"{super().describe()}, duration={self.duration_minutes} minutes"

## 4) Non‑circulating subclass

Create `ReferenceBook(Book)` that **cannot** be checked out. Override `loan_period_days()` to return `0`. In `LoanDesk.checkout`, demonstrate a guard that raises `LibraryError('non-circulating')` if the period is 0.

In [16]:
# Your code here

from dataclasses import dataclass
from datetime import datetime, timedelta
from typing import List, Dict, Optional

@dataclass
class Book:
    isbn: str
    title: str
    copies: int = 1

    def loan_period_days(self) -> int:
        """Default loan period for a generic book."""
        return 14

    def describe(self) -> str:
        return f"Book<{self.isbn}>: {self.title} (copies={self.copies})"
    
class ReferenceBook(Book):
    def loan_period_days(self) -> int:
        return 0 

class LibraryError(Exception):
    pass

class Catalog:
    def __init__(self):
        self._books: Dict[str, Book] = {}

    def add_book(self, book: Book) -> None:
        if book.isbn in self._books:
            raise DuplicateBookError(f"ISBN already exists: {book.isbn}")
        if book.copies < 0:
            raise ValueError("copies must be non-negative")
        self._books[book.isbn] = book

    def get_book(self, isbn: str) -> Optional[Book]:
        return self._books.get(isbn)


@dataclass
class Member:
    member_id: str
    email: str

    def max_concurrent_loans(self) -> int:
        return 5

    def describe(self) -> str:
        return f"Member<{self.member_id}>"

@dataclass
class Loan:
    isbn: str
    member_id: str
    due_date: datetime
    returned: bool = False

    def mark_returned(self) -> None:
        self.returned = True
    

    

class LoanDesk:
    """Very small service; deliberately simple for Week 8 examples."""
    def __init__(self, catalog: Catalog):
        self.catalog = catalog
        self.loans: List[Loan] = []

    def checkout(self, member: Member, book: Book) -> Loan:
        if book.loan_period_days() == 0:
            raise LibraryError("book is non-circulating")
        # naive stock check
        if book.copies <= 0:
            raise LibraryError("no available copies")
        book.copies -= 1
        due = datetime.now() + timedelta(days=book.loan_period_days())
        loan = Loan(isbn=book.isbn, member_id=member.member_id, due_date=due)
        self.loans.append(loan)
        return loan

    def checkin(self, loan: Loan) -> None:
        if not loan.returned:
            b = self.catalog.get_book(loan.isbn)
            if b:
                b.copies += 1
            loan.mark_returned() 

# Update/extend LoanDesk.checkout minimally to guard non‑circulating

## 5) Member specialization

Create `Student(Member)` and `Staff(Member)`. Students can have 5 concurrent loans; staff 10. Override `max_concurrent_loans()` accordingly.

In [17]:
# Your code here
class Student(Member):
    def max_consurrent_loans(self) -> int:
        return 5
class Staff(Member):
    def max_concurrent_loans(self) -> int:
        return 10

## 6) Polymorphic fine calculation

Write a function `late_fee(book: Book, days_late: int) -> float` that uses polymorphic behavior:
- PrintedBook: $0.25/day
- EBook: $0.10/day
- AudioBook: $0.15/day
- Fallback (Book): $0.20/day
Use `isinstance` checks only; do not modify the classes for this one.

In [20]:
# Your code here
class PrintedBook(Book):
    pass
class EBook(Book):
    pass
class AudioBook(Book):
    pass
def late_fee(book: Book, days_late: int) -> float:
    if isinstance(book, PrintedBook):
        return days_late * 0.25
    elif isinstance(book, EBook):
        return days_late * 0.10
    elif isinstance(book, AudioBook):
        return days_late * 0.15
    else:
        return days_late * 0.20

## 7) Polymorphism without isinstance

Refactor your approach so **each subclass** implements `daily_late_fee()` and `late_fee(days_late)` (calling `daily_late_fee()`), then write a single function `compute_fee(book: Book, days_late: int)` that calls `book.late_fee(days_late)` without type checks.

In [22]:
# Your code here
class Book:
    def daily_late_fee(self) -> float:
        return 0.20
    def late_fee(self, days_late: int) -> float:
        return days_late * self.daily_late_fee()
class PrintedBook(Book):
    def daily_late_fee(self) -> float:
        return 0.25
class EBook(Book):
    def daily_late_fee(self) -> float:
        return 0.10
class AudioBook(Book):
    def daily_late_fee(self) -> float:
        return 0.15
# Add methods to Book and subclasses; avoid breaking earlier exercises if possible.
def compute_fee(book: Book, days_late: int) -> float:
   return book.late_fee(days_late)
    

## 8) Overriding __str__

Override `__str__` in `Book` to return `<Book isbn=... title=...>`. Override it in one subclass to include subtype info, e.g., `<PrintedBook isbn=...>`.

In [23]:
# Your code here
# Override in Book and one subclass
class Book:
    def __init__(self, isbn: str, title: str):
        self.isbn = isbn
        self.title = title
        
    def __str__(self) -> str:
        return f"<Book isbn={self.isbn} title={self.title}>"

class PrintedBook(Book):
    def __str__(self) -> str:
        return f"<PrintedBook isbn={self.isbn} title={self.title}>"

## 9) Composition vs. inheritance

Create a small `Notifier` class with method `notify(member: Member, message: str)`. Demonstrate **composition** by adding a `notifier` attribute to `LoanDesk` and using it during checkout to acknowledge a loan. Keep `Notifier` very simple (e.g., print or collect messages).

In [24]:
# Your code here
class Notifier:
    def notify(self, member: Member, message: str):
        print(f"Notify {member}: {message}")

class LoanDesk:
    def __init__(self, notifier: Notifier):
        self.notifier = notifier

    def checkout(self, member: Member, book: Book):
        self.notifier.notify(member, f"You have checked out {book.title}")

# Integrate into LoanDesk via composition (not inheritance)

## 10) Enforcing limits polymorphically

Modify `LoanDesk.checkout` to check a member's current active loans (for that member_id) and compare to `member.max_concurrent_loans()` before allowing checkout. Demonstrate with a `Student` hitting the limit and a `Staff` not hitting it.

In [33]:
# Your code here
from dataclasses import dataclass
from datetime import datetime, timedelta
from typing import List, Dict, Optional

@dataclass
class Book:
    isbn: str
    title: str
    copies: int = 1

    def loan_period_days(self) -> int:
        return 14
    
@dataclass  
class Member:
    member_id: str
    email: str

    def max_concurrent_loans(self) -> int:
        return 5


@dataclass
class Student(Member):
    def max_concurrent_loans(self) -> int:
        return 2
    
@dataclass
class Staff(Member):
    def max_concurrent_loans(self) -> int:
        return 10

class Loan:
    def __init__(self, isbn: str, member_id: str, due_date: datetime):
        self.isbn = isbn
        self.member_id = member_id
        self.due_date = due_date
        self.returned = False

class LibraryError(Exception):
    pass

class Catalog:
    def __init__(self):
        self.books: Dict[str, Book] = {}

    def get_book(self, isbn: str) -> Optional[Book]:
            return self.books.get(isbn)

    
class LoanDesk:
    def __init__(self, catalog: Catalog):
        self.catalog = catalog
        self.loans: List[Loan] = []

    def checkout(self, member: Member, book: Book) -> Loan:
        active_loans = [
            loan for loan in self.loans 
            if loan.member_id == member.member_id and not loan.returned
        ]
        if len(active_loans) >= member.max_concurrent_loans():
            raise LibraryError("Loan limit reached")
        
        if book.copies <= 0:
            raise LibraryError("no available copies")
        book.copies -= 1
        due = datetime.now() + timedelta(days=book.loan_period_days())
        loan = Loan(isbn=book.isbn, member_id=member.member_id, due_date=due)
        self.loans.append(loan)
        return loan
    
    def checkin(self, loan: Loan) -> None:
        if not loan.returned:
            b = self.catalog.get_book(loan.isbn)
            if b:
                b.copies += 1
            loan.mark_returned()
        
   

# Update LoanDesk.checkout and show a brief demo

if __name__ == "__main__":
    catalog = Catalog()
    catalog.books = {"1": Book(isbn="1", title="Short Book")}

    student = Student(member_id="s1", email="s1.com")
    desk = LoanDesk(catalog)

    desk.checkout(student, catalog.get_book("1")) 

    try:
        desk.checkout(student, catalog.books["1"])
    except LibraryError as e:
        print(e)    


    catalog.books["1"].copies += 1

    staff = Staff(member_id="stafft1", email="staff1.com")
    try:
        desk.checkout(staff, catalog.get_book("1"))
        print("Staff checkout successful")
    except LibraryError as e:
        print(e)

no available copies
Staff checkout successful


## 11) Subclass‑specific behavior

Add a method `download_link()` to `EBook` returning a fake URL string using the ISBN. Do not add this to `Book` or other subclasses. Show a short snippet where you use duck typing safely by checking `hasattr` before calling.

In [34]:
# Your code here
# Add method to EBook and a usage demo with duck typing

from dataclasses import dataclass

@dataclass
class Book:
    isbn: str
    title: str

@dataclass
class EBook(Book):
    def download_link(self) -> str:
        return f"http://book.com/electronic/{self.isbn}"
    
def download(book: Book):
    if hasattr(book, 'download_link'):
        print(book.download_link())
    else:
        print("Can't be downloaded")

if __name__ == "__main__":
    eb = EBook(isbn="123", title="E-Book Title")
    download(eb)  # prints the download link

    b = Book(isbn="456", title="Regular Book")
    download(b)   # prints "Can't be downloaded"

http://book.com/electronic/123
Can't be downloaded


## 12) Polymorphic loan period by member type

Some libraries extend loan periods for `Staff`. Implement `effective_loan_period(book: Book, member: Member) -> int`:
- start from `book.loan_period_days()`
- if `isinstance(member, Staff)`, add +7 days
Return the resulting days.

In [36]:
# Your code here

from dataclasses import dataclass

@dataclass
class Book:
    isbn: str
    title: str

    def loan_period_days(self) -> int:
        return 14
    
@dataclass
class Member:
    member_id: str
    email: str

    def max_concurrent_loans(self) -> int:
        return 5
    
@dataclass
class Student(Member):
    def max_concurrent_loans(self) -> int:
        return 2
    
@dataclass
class Staff(Member):
    def max_concurrent_loans(self) -> int:
        return 10


def effective_loan_period(book: Book, member: Member) -> int:
    days = book.loan_period_days()
    if isinstance(member, Staff):
        days += 7
    return days

if __name__ == "__main__":
    book = Book(isbn="111", title="Intro to Python")
    student = Student(member_id="s1", email="s1.gmail.com")
    staff = Staff(member_id="st1", email="st1.gmail.com")
     

## 13) Override equality semantics (dataclass)

For `Book`, override `__eq__` so that books are considered equal iff ISBNs match (ignore title/copies). Write quick tests comparing a `PrintedBook` and `EBook` with the same ISBN—they should be equal by ISBN.

In [42]:
# Your code here
# Override __eq__ on Book carefully; demonstrate with examples

from dataclasses import dataclass

@dataclass(eq=False)
class Book:
    isbn: str
    title: str
    
    def __eq__(self, other):
        if isinstance(other, Book):
            return self.isbn == other.isbn
        return NotImplemented

@dataclass
class PrintedBook(Book):
    copies: int = 1

@dataclass
class EBook(Book):
    def download_link(self) -> str:
        return f"http://book.com/electronic/{self.isbn}"
    
book1 = PrintedBook(isbn="123", title="Python 101", copies=3)
book2 = EBook(isbn="123", title="Python 101")   
book3 = Book(isbn="456", title="Java 101")

book1 ==book2 
book1 == book3  
        

False

## 14) Draft a small class hierarchy diagram (markdown)

In **markdown**, sketch a tiny hierarchy diagram for `Book <- PrintedBook | EBook | AudioBook | ReferenceBook` and `Member <- Student | Staff`. No code—just a clear diagram using text/ASCII.

In [53]:
# (Write your diagram in this markdown cell by editing it after running the notebook.)

Book
  ├── PrintedBook
  ├── EBook
  ├──AudioBook
  └── ReferenceBook


Member
  ├── Student
  └── Staff

IndentationError: unexpected indent (1383472465.py, line 4)

## 15) Replace conditional with polymorphism

Currently, `LoanDesk.checkout` always uses `book.loan_period_days()`. Add an overridable method `checkout_message()` to `Book` and override it in at least two subclasses to customize the user‑facing message returned by `LoanDesk.checkout` (e.g., 'Enjoy your audiobook!'). Show the different messages without `if/elif` chains.

In [44]:
# Your code here
# Add checkout_message to Book and override in subclasses; update LoanDesk.checkout to use it

from dataclasses import dataclass
from typing import List, Dict, Optional

@dataclass
class Book:
    isbn: str
    title: str
    
    def loan_period_days(self) -> int:
        return 14

    def checkout_message(self) -> str:
        return f"Enjoy reading '{self.title}'!"
    
@dataclass
class AudioBook(Book):
    def checkout_message(self) -> str:
        return f"Enjoy your audio book '{self.title}'!"
    
@dataclass
class PrintedBook(Book):
    def checkout_message(self) -> str:
        return f"Enjoy your printed book '{self.title}'!"
    
class Member:
    def __init__(self, member_id: str, email: str):
        self.member_id = member_id
        self.email = email

    def max_concurrent_loans(self) -> int:
        return 5
    
@dataclass
class Loan:
    isbn: str
    member_id: str
    returned: bool = False

class Catalog:
    def __init__(self):
        self.books ={}
    def get_book(self, isbn: str) -> Book:
        return self.books.get(isbn)
    
class LoanDesk:
    def __init__(self, catalog: Catalog):
        self.catalog = catalog
        self.loans: List[Loan] = []

    def checkout(self, member: Member, book: Book) -> str:
        loan = Loan(isbn=book.isbn, member_id=member.member_id)
        self.loans.append(loan)
        return book.checkout_message()
    
    

## 16) Subclass‑specific stock policy

Override `LoanDesk.checkout` to deny checkout of `EBook` if `copies < 0` (simulate licensing depletion), but allow `PrintedBook` as long as `copies > 0`. Implement this by relying on each subclass's own `can_checkout(stock: int) -> bool` method. Default in `Book` should be `stock > 0`.

In [45]:
# Your code here
# Add can_checkout to Book and subclasses; update LoanDesk.checkout accordingly

from dataclasses import dataclass
from typing import List, Dict, Optional

@dataclass
class Book:
    isbn: str
    title: str
    copies: int = 1
    
    def can_checkout(self, stock: int) -> bool:
        return stock > 0
    
@dataclass
class PrintedBook(Book):
    pass

@dataclass
class EBook(Book):
    def can_checkout(self, stock: int) -> bool:
        return stock >= 0
    
@dataclass
class Loan:
    isbn: str
    member_id: str
    returned: bool = False

class Member:
    def __init__(self, member_id: str, email: str):
        self.member_id = member_id
        self.email = email

class Catalog:
    def __init__(self):
        self.books ={}
    def get_book(self, isbn: str) -> Book:
        return self.books.get(isbn)
    
class LoanDesk:
    def __init__(self, catalog: Catalog):
        self.catalog = catalog
        self.loans: List[Loan] = []

    def checkout(self, member: Member, book: Book) -> bool:
        if not book.can_checkout(book.copies):
            return False
        book.copies -= 1
        self.loans.append(Loan(isbn=book.isbn, member_id=member.member_id))
        return True


## 17) Sorting polymorphically

Create a list mixing `PrintedBook`, `EBook`, and `AudioBook`. Implement a function `sort_books_for_display(books: list[Book]) -> list[Book]` that sorts by this precedence: Printed first, then EBook, then AudioBook; ties broken by title. Use a key function that relies on `isinstance` or a small polymorphic `display_rank()` method.

In [46]:
# Your code here
from dataclasses import dataclass
from typing import List, Dict, Optional

@dataclass
class Book:
    isbn: str
    title: str

    def display_rank(self) -> int:
        return 3
    
@dataclass
class PrintedBook(Book):
    def display_rank(self) -> int:
        return 0

@dataclass
class EBook(Book):
    def display_rank(self) -> int:
        return 1
    
@dataclass
class AudioBook(Book):
    def display_rank(self) -> int:
        return 2

def sort_books_for_display(books: list[Book]) -> list[Book]:
    return sorted(books, key=lambda b: b.display_rank())
    

## 18) Minimal polymorphic report

Write `summarize_books(books: list[Book]) -> list[str]` that returns `describe()` for each. Show that the correct overridden `describe()` is used without `if/elif`.

In [47]:
# Your code here
from dataclasses import dataclass
from typing import List, Dict, Optional

@dataclass
class Book:
    isbn: str
    title: str

    def describe(self) -> str:
        return f"Book: {self.title}"
    
@dataclass
class PrintedBook(Book):
    def describe(self) -> str:
        return f"Printed: {self.title}"
    
@dataclass
class EBook(Book):
    def describe(self) -> str:
        return f"EBook: {self.title}"
    
@dataclass
class AudioBook(Book):
    def describe(self) -> str:
        return f"AudioBook: {self.title}"
    
def summarize_books(books: list[Book]) -> list[str]:
    return [book.describe() for book in books]

## 19) Unit test: overriding works

Using `unittest`, add a small test class that checks `loan_period_days()` for `PrintedBook` (21) and `ReferenceBook` (0), and that `__str__` includes the subclass name for one subtype.

In [49]:
# Your code here
import unittest

class TestWeek8Inheritance(unittest.TestCase):
    def test_loan_periods_and_str(self):
        printed = PrintedBook(isbn = "111", title = "Printed Title")
        reference = ReferenceBook(isbn = "222", title = "Reference Title")

        self.assertEqual(printed.loan_period_days(), 21)
        self.assertEqual(reference.loan_period_days(), 0)

        self.assertIn("Printed", str(printed))
        self.assertIn("Reference", str(reference))

# # To run inside notebook:
# # unittest.main(argv=['-v'], exit=False)

## 20) Polymorphic fee scenario (end‑to‑end)

Create a short demo that:
- Builds a `Catalog` and `LoanDesk` (with `Notifier` if you implemented it)
- Adds one book of each subtype
- Checks each out to a `Student`
- Simulates `days_late` values and prints fees using your polymorphic fee API
Show that different subtypes yield different fees without `if/elif` at the call site.

In [54]:
# Your code here
# End-to-end demo using your polymorphic methods

from dataclasses import dataclass
from typing import List, Dict, Optional

@dataclass
class Book:
    isbn: str
    title: str

    def loan_period_days(self) -> int:
        return 14

    def checkout_message(self) -> str:
        return f"Enjoy reading '{self.title}'!"
    
    def display_rank(self) -> int:
        return 3
    
    def describe(self) -> str:
        return f"Book: {self.title}"
    
    def late_fee(self, days_late: int) -> float:
        return days_late * 0.5
    
@dataclass
class PrintedBook(Book):
    def loan_period_days(self) -> int:
        return 21

    def checkout_message(self) -> str:
        return f"Enjoy your printed book '{self.title}'!"

    def display_rank(self) -> int:
        return 0

    def describe(self) -> str:
        return f"PrintedBook: {self.title}"

    def late_fee(self, days_late: int) -> float:
        return days_late * 0.25
    
@dataclass
class EBook(Book):
    def loan_period_days(self) -> int:
        return 14

    def checkout_message(self) -> str:
        return f"Enjoy your ebook '{self.title}'!"

    def display_rank(self) -> int:
        return 1

    def describe(self) -> str:
        return f"EBook: {self.title}"

    def late_fee(self, days_late: int) -> float:
        return days_late * 1.0
    

@dataclass
class AudioBook(Book):
    def checkout_message(self) -> str:
        return f"Enjoy your audiobook '{self.title}'!"
    def display_rank(self) -> int:
        return 2
    def describe(self) -> str:
        return f"AudioBook: {self.title}"
    def late_fee(self, days_late: int) -> float:
        return days_late * 0.75

    
@dataclass
class Loan:
    isbn: str
    member_id: str
    returned: bool = False

class Member:
    def __init__(self, member_id: str, email: str):
        self.member_id = member_id
        self.email = email
    
    def max_concurrent_loans(self) -> int:
        return 5
    
class Catalog:
    def __init__(self):
        self.books: Dict[str, Book] = {}

    def get_book(self, isbn: str) -> Optional[Book]:
        return self.books.get(isbn)
    
class LoanDesk:
    def __init__(self, catalog: Catalog):
        self.catalog = catalog
        self.loans: List[Loan] = []

    def checkout(self, member: Member, book: Book) -> str:
        loan = Loan(isbn=book.isbn, member_id=member.member_id)
        self.loans.append(loan)
        return book.checkout_message()
    
def sort_books_for_display(books: List[Book]) -> List[Book]:
    return sorted(books, key=lambda b: (b.display_rank(), b.title))

def summarize_books(books: List[Book]) -> List[str]:
    return [book.describe() for book in books]

def fee_demo():
    catalog = Catalog()
    desk = LoanDesk(catalog)
    catalog.books = {
        "1": PrintedBook(isbn="1", title="Printed Book"),
        "2": EBook(isbn="2", title="E-Book"),
        "3": AudioBook(isbn="3", title="Audio Book"),
    }
    student = Member(member_id="s1", email="s1.gmail.com")

    for book in catalog.books.values():
        desk.checkout(student, book)

    day_late = 3
    results = {}
    for book in catalog.books.values():
        fee = book.late_fee(day_late)
        results[book.title] = fee
    return results




## Python skills you'll need (Weeks 1–8)

- **Core syntax & data types:** variables, strings, numbers, booleans
- **Collections:** lists, dicts (basic use), simple list/dict comprehensions
- **Control flow:** `if/elif/else`, `for`, `while`
- **Functions & modules:** defining functions, parameters, returns, imports
- **File I/O & JSON (basic):** open/read/write, simple JSON usage
- **Classes & objects (Weeks 4–6):** defining classes, attributes, methods, `__init__`, `__str__`
- **Encapsulation basics:** simple validation; naming conventions for "private" attributes
- **Methods:** instance/class/static methods (as introduced up to Week 6)
- **Error handling & testing (Week 7):** `try/except/else/finally`, custom exceptions, basic `unittest`
- **Week 8 focus:** **single inheritance**, **method overriding**, **`super()`**, **polymorphism via common methods**, and **composition vs. inheritance** decisions
- **Standard library familiarity:** `datetime`, `timedelta`, built‑in exceptions
